# IoT Analysis for a Hydroponic Vertical Farm: Data Processing

A friend of mine has been developing a hydroponic vertical farm prototype in his apartment in River North, with the intention to set up an operational system in the basement of his family home in the north suburbs of Chicago, where he will grow and sell microgreens. As part of this project, he has asked for my help doing two things:

1. Modeling the interactions between the different components of the system to facilitate optimal crop yield, and
2. Developing a live dashboard to remotely monitor and control the system.  

In this notebook, I will detail the data processing pipeline I've developed to use for both of these tasks. I will be adding new notebooks to this series (and updating this one) as the project progresses.

### Reading and parsing the data

The data streams for this project include sensors that measure and report the temperature, pH, and conductivity of the hydroponic system, as well as the status of the pumps and fans that control the system. These streams are received in multiple feeds of one-dimensional text data, which must be parsed, interpreted, and tidied before any analysis can be done. The data streams from two separate feeds, labeled "ESP32" and "Arduino". The first step is to combine them into a single DataFrame object.

**NOTE**: As the system is still being prototyped, the data currently used in this notebook is in read in as a .xls file. Future versions will use requests for live data via REST API.

In [1]:
# package imports
import pandas as pd
import numpy as np

# read raw data to dict of dataframe objects, then concatenate them together
file = 'C:\\Users\\Alex\\Projects\\Rosies_Farm\\pHDoseUpExperiment_12142022.xls'
raw_data = pd.read_excel(file, sheet_name=None, header=None)
raw_data = pd.concat([raw_data['ESP32'], raw_data['Arduino']])
display(raw_data.head())

,0
0,12-10-2022 13:45:45.133 -> Water Temp: 28.50
1,12-10-2022 13:45:50.150 -> Water Temp: 28.50
2,12-10-2022 13:45:55.163 -> Water Temp: 28.50
3,12-10-2022 13:45:58.926 -> <Relay:0:0:0>
4,12-10-2022 13:46:00.134 -> Water Temp: 28.50


Both feeds exhibit the format of `"{timestamp}->{value}"`; the `value` portion is inconsistent between all the sensors, but the timestamp can be parsed out easily using the `->` string as a delimiter. 

In [2]:
data = raw_data.iloc[:,0].str.split('->', expand=True)
data.columns = 'Time', 'Reading'
display(data)

,Time,Reading
0,12-10-2022 13:45:45.133,Water Temp: 28.50
1,12-10-2022 13:45:50.150,Water Temp: 28.50
2,12-10-2022 13:45:55.163,Water Temp: 28.50
3,12-10-2022 13:45:58.926,<Relay:0:0:0>
4,12-10-2022 13:46:00.134,Water Temp: 28.50
...,...,...
32780,12-11-2022 00:18:36.435,Scale Reading: 72.45
32781,12-11-2022 00:18:38.851,Received Chars: 99:R
32782,12-11-2022 00:18:38.851,Channel: 99
32783,12-11-2022 00:18:38.851,CMD: R


Next, we define a function to be applied row-wise to the dataframe which will parse and interpret the different sensor and pump outputs based on their specific patterns and ID codes. In particular, the ID codes for the rows with the "Relay" tag follow the pattern *X:Y:Z* where *X:Y* can be used to identify the relay, and *Z* is the binary value reporting the state of the relay.  

Additionally, several types of readings which carry no useful information are now filtered out.

In [3]:
def process_row(row):
    '''
    Function for use with pd.apply(). Parses sensor outputs into
    Measurement and Value.
    '''
    
    # parse and clean up the reported reading
    measurement, value = row.Reading.split(':', 1)
    measurement = measurement.strip(' <')
    value = value.strip('>')
    
    # store relay codes in dict
    relay_ids = {
        '0:0': 'Mix Pump',
        '0:1': 'Main Pump',
        '1:0': 'Mixing Fan'
    }
    
    # interpret relay codes, where present
    if measurement == 'Relay':
        relay = relay_ids[value[:3]]
        measurement = relay
        value = value[-1]
    
    # parse Dosing measurement, with its unique format
    elif measurement == 'Dosing':
        value = value.split(':')[1]
    
    # filter out unwanted readings
    if measurement in ('Received Chars', 'CMD', 'Channel', '') or value == '':
        return np.nan, np.nan, np.nan # returning a row of NaNs allows us to drop these rows instantly using .dropna()
    
    # return parsed row
    else:
        return [row.Time, measurement, value]

    
# apply new fucntion row-wise, dropping unwanted rows and resetting index to account for those dropped
data = data.apply(process_row, axis=1, result_type='expand').dropna().reset_index(drop=True)
data.columns = 'Time', 'Measurement', 'Value'

# recast to proper dtypes
data = data.astype(
    {
        'Time': np.datetime64,
        'Value': float
    }
)

display(data)

,Time,Measurement,Value
0,2022-12-10 13:45:45.133,Water Temp,28.500
1,2022-12-10 13:45:50.150,Water Temp,28.500
2,2022-12-10 13:45:55.163,Water Temp,28.500
3,2022-12-10 13:45:58.926,Mix Pump,0.000
4,2022-12-10 13:46:00.134,Water Temp,28.500
...,...,...,...
21379,2022-12-11 00:18:29.857,PH,8.701
21380,2022-12-11 00:18:30.693,Scale Reading,72.450
21381,2022-12-11 00:18:34.859,EC,373.600
21382,2022-12-11 00:18:36.435,Scale Reading,72.450


### Pivoting and Resampling

Now that everything is parsed, we can apply the `pivot_table` DataFrame method, specifying `columns='Measurement', values='Value', 'index='Time'` so that all of the different sensors, pumps, and fans in the `Measurement` field get their own column that is populated with the `Value` of that measurement at a given `Time`, which is set as the index.

In [4]:
# pivot so measurements are columns
data = data.pivot_table(columns='Measurement', values='Value', index='Time')

# rename semantically irritating columns
data.rename(columns={
    'Dosing':'pH Up Dosing',
    'EC': 'Conductivity',
    'PH': 'pH'}, inplace=True)


display(data)
display(data.info())

Measurement,pH Up Dosing,Conductivity,Main Pump,Mix Pump,Mixing Fan,pH,Scale Reading,Water Temp
Time,,,,,,,,
2022-12-10 13:45:45.133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.50
2022-12-10 13:45:46.169,NaN,NaN,NaN,NaN,NaN,7.518,NaN,NaN
2022-12-10 13:45:46.685,NaN,NaN,NaN,NaN,NaN,NaN,72.04,NaN
2022-12-10 13:45:50.150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.50
2022-12-10 13:45:51.141,NaN,355.9,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-12-11 00:18:33.828,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.44
2022-12-11 00:18:34.859,NaN,373.6,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-11 00:18:36.435,NaN,NaN,NaN,NaN,NaN,NaN,72.45,NaN


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 21203 entries, 2022-12-10 13:45:45.133000 to 2022-12-11 00:18:39.827000
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   pH Up Dosing   34 non-null     float64
 1   Conductivity   3481 non-null   float64
 2   Main Pump      119 non-null    float64
 3   Mix Pump       335 non-null    float64
 4   Mixing Fan     107 non-null    float64
 5   pH             3482 non-null   float64
 6   Scale Reading  6181 non-null   float64
 7   Water Temp     7596 non-null   float64
dtypes: float64(8)
memory usage: 1.5 MB


None

Now that every row has been expanded we can see that we have many NaN values due to the sensors not being perfectly synchronized and the time index being recorded to the millisecond. The dataframe may only take up 1.5MB now, but as the system scales this will quickly become a problem. Since we know that the shortest measurement interval (i.e. time between readings) for any of the sensors is 5 seconds, we can use the `resample` dataframe method to collapse our data into 5 second intervals.

In [5]:
# resample to minimize sparseness of the data
data = data.resample('5s').mean()
display(data)
display(data.info())

Measurement,pH Up Dosing,Conductivity,Main Pump,Mix Pump,Mixing Fan,pH,Scale Reading,Water Temp
Time,,,,,,,,
2022-12-10 13:45:45,NaN,NaN,NaN,NaN,NaN,7.518,72.04,28.50
2022-12-10 13:45:50,NaN,355.9,NaN,NaN,NaN,NaN,72.04,28.50
2022-12-10 13:45:55,NaN,NaN,NaN,0.0,NaN,7.516,72.04,28.50
2022-12-10 13:46:00,NaN,NaN,NaN,NaN,NaN,NaN,72.04,28.50
2022-12-10 13:46:05,NaN,357.6,NaN,NaN,NaN,NaN,NaN,28.50
...,...,...,...,...,...,...,...,...
2022-12-11 00:18:15,NaN,NaN,NaN,NaN,NaN,8.697,72.45,24.44
2022-12-11 00:18:20,NaN,373.5,NaN,NaN,NaN,NaN,NaN,24.44
2022-12-11 00:18:25,NaN,NaN,NaN,NaN,NaN,8.701,72.46,24.44


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7595 entries, 2022-12-10 13:45:45 to 2022-12-11 00:18:35
Freq: 5S
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   pH Up Dosing   24 non-null     float64
 1   Conductivity   3481 non-null   float64
 2   Main Pump      64 non-null     float64
 3   Mix Pump       182 non-null    float64
 4   Mixing Fan     54 non-null     float64
 5   pH             3482 non-null   float64
 6   Scale Reading  6181 non-null   float64
 7   Water Temp     7578 non-null   float64
dtypes: float64(8)
memory usage: 534.0 KB


None

Great! At **534KB**, we've cut the required memory to just over a third of the original. 

## Handling Missing Values

There are still many NaNs, which we can now handle via forward filling. We exclude the Dosing column when forward filling due to the nature of the measurement - with the other sensors, it is safe to assume that the value doesn't change in between reported readings (for example, if the pH sensor sends a reading of 7.9 then it makes sense to assume the pH stays at 7.9 until the next time we hear from the sensor). Likewise, the other pumps and fans are reporting a *change in state*, which will be the same until the state changes again.  

However, a report from the pH Up Dosing pump is an indication that *something was done* to the system (in this case, that some amount of pH Up solution was added). As that is something that depends on knowing **when** the value is reported, which might be obscured by the fact that the value will likely be the same most of the time as Dosing remains consistent, we leave the NaNs in the Dosing column as is for now.

In [6]:
# forward fill all NaNs except the Dosing column
data.loc[:, data.columns!='Dosing'] = data.loc[:, data.columns!='Dosing'].fillna(method='ffill')

In [7]:
display(data)
display(data.info())

Measurement,pH Up Dosing,Conductivity,Main Pump,Mix Pump,Mixing Fan,pH,Scale Reading,Water Temp
Time,,,,,,,,
2022-12-10 13:45:45,NaN,NaN,NaN,NaN,NaN,7.518,72.04,28.50
2022-12-10 13:45:50,NaN,355.9,NaN,NaN,NaN,7.518,72.04,28.50
2022-12-10 13:45:55,NaN,355.9,NaN,0.0,NaN,7.516,72.04,28.50
2022-12-10 13:46:00,NaN,355.9,NaN,0.0,NaN,7.516,72.04,28.50
2022-12-10 13:46:05,NaN,357.6,NaN,0.0,NaN,7.516,72.04,28.50
...,...,...,...,...,...,...,...,...
2022-12-11 00:18:15,0.0,373.2,0.0,1.0,0.0,8.697,72.45,24.44
2022-12-11 00:18:20,0.0,373.5,0.0,1.0,0.0,8.697,72.45,24.44
2022-12-11 00:18:25,0.0,373.5,0.0,1.0,0.0,8.701,72.46,24.44


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7595 entries, 2022-12-10 13:45:45 to 2022-12-11 00:18:35
Freq: 5S
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   pH Up Dosing   6836 non-null   float64
 1   Conductivity   7594 non-null   float64
 2   Main Pump      7569 non-null   float64
 3   Mix Pump       7593 non-null   float64
 4   Mixing Fan     6837 non-null   float64
 5   pH             7595 non-null   float64
 6   Scale Reading  7595 non-null   float64
 7   Water Temp     7595 non-null   float64
dtypes: float64(8)
memory usage: 534.0 KB


None

We could handle the remaining missing values at the beginning of the dataset, but as this pipeline is meant to be implemented in a continuous operation setting, it won't be of much use to do so. If it becomes necessary to handle these values when modeling, we will cross that bridge when we come to it.

## Next Steps

Now that our pipeline is in place, the next order of business will be to set up some visualizations for certain key system properties like pH and Conductivity. As the data used in this notebook at the time of writing was collected for the purpose of modeling the impact that pH Up Dosing has on Conductivity, we will explore that relationship through these visualizations and also be preparing our data for time-series forecasting.